# Clean up Louisiana Ownership Data

The Louisiana Department of Health provided two spreadsheets containing ownership information on two separate dates. (See the README in the ownership data directory for more information.) This notebook combines that information with the facility data scraped from QCOR.

In [1]:
import pandas as pd

## Load Louisiana DPH spreadsheets

We received these records from two data requests to the Louisiana Department of Public Health

In [2]:
RAW_OWNERSHIP_DIR = "../../data/ownership/raw/"
INTERMEDIATE_OWNERSHIP_DIR = "../../data/ownership/intermediate/"

In [3]:
la_original = pd.read_excel(RAW_OWNERSHIP_DIR + "Data Request JT 08122021 rev.xlsx", dtype={"ZIP CODE": str})

In [4]:
la_historical = (
    pd
    .read_excel(
        RAW_OWNERSHIP_DIR + "Data_Request_JT_12072021.xlsx", 
        dtype={"GEOGRAPHICAL_ZIP": str}
    )
    .rename(
        columns={
            "GEOGRAPHICAL_ZIP": "ZIP CODE", 
            "FACILITY_NAME": "FACILITY NAME"
        }
    )
)

In [5]:
len(la_original)

519

In [6]:
len(la_historical)

60

The Louisiana state data does not include CMS IDs, so we have to join on name.

In [7]:
name_spreadsheet = pd.read_csv(INTERMEDIATE_OWNERSHIP_DIR + "la-facility-match.csv") # This is a manual cleaner

def make_capped_name(name):
    CAPPED_NAME = name.upper().replace(".", "")
    if CAPPED_NAME in name_spreadsheet["state_name"].values:
        return name_spreadsheet.set_index("state_name").loc[CAPPED_NAME]["qcor_name"]
    else:
        return CAPPED_NAME

In [8]:
zip_fix = {
    "PINECREST SUPPORTS AND SERVICES CENTER": "71361",
    "HARMONY CENTER NO 2": "70802",
    "ST ROSALIE GROUP HOME": "70053",
    "G B COOLEY SHANNON GROUP HOME": "71201",
    "G B COOLEY SPURGEON GROUP HOME": "71202",
    "G B COOLEY MOORE-WEATHERFORD GROUP HOME": "71201",
    "G B COOLEY TENNESSEE GROUP HOME": "71203",
    "PLANTATION COMMUNITY HOME": "70810",
    "HEYMAN LANE COMMUNITY HOME": "71303",
    "4-B GROUP HOME": "71282",
    "ABBEVILLE GROUP HOME": "70511",
    "ALPHA HOUSE": "70404",
    "CANE PLACE COMMUNITY HOME": "70506",
    "CHARLES P GRECO HOME": "71306",
    "GADSDEN COMMUNITY HOME": "70062",
    "JACKSON GROUP HOME": "71303",
    "MEADOWBROOK COMMUNITY HOME": "71105",
    "MELBROOK COMMUNITY HOME": "70056",
    "MUSTARD SEED GROUP HOME": "70433",
    "NEWCASTLE COMMUNITY HOME": "70714",
    "NORBEN DRIVE GROUP HOME": "70605",
    "RANGE ROAD COMMUNITY HOME": "70401",
    "SUMMER HOUSE": "70726",
    "VERNI COMMUNITY HOME": "71306",
    "WILLOWICK GROUP HOME": "70607",
    "OCEAN AVENUE COMMUNITY HOME": "70114",
    "AUBURN COMMUNITY HOME": "70065",
    "G B COOLEY WALNUT GROUP HOME": "71202",
    "NORTH MITCHELL STREET HOME": "70664",
    "PONDEROSA HOME": "70815",
    "ST ROSALIE GROUP HOME": "70131"
}

In [9]:
state_data = (
    pd
    .concat([ la_original, la_historical ])
    .assign(capped_name = lambda df: df["FACILITY NAME"].apply(lambda x: make_capped_name(x)))
    .assign(
        zip_code = lambda df: df
        .apply(
            lambda x: zip_fix[x["capped_name"]] if x["capped_name"] in zip_fix.keys() else x["ZIP CODE"], 
            axis=1
        )
    )
)

In [10]:
state_data.head()

,FACILITY NAME,ADDRESS,CITY,STATE,ZIP CODE,PARISH,ADMINISTRATOR,EMAIL_ADDRESS,Number of Licened Bedrooms,Number of Licensed Beds,...,NAME_OF_OWNER_2,NAME_OF_OWNER_3,NAME_OF_OWNER_4,NAME_OF_CORPORATION,GEOGRAPHICAL_STREET,GEOGRAPHICAL_CITY,GEOGRAPHICAL_STATE,Number of Licensed Bedrooms,capped_name,zip_code
0,1000 Milne Circle,1000 Milne Circle,COVINGTON,LA,70435,ST. TAMMANY,Jodie Robinson,jrobinson@milnegh.com,4.0,4,...,NaN,NaN,NaN,"ALEXANDER MILNE DEVELOPMENTAL SERVICES, INC.",NaN,NaN,NaN,NaN,1000 MILNE CIRCLE,70435
1,1004 Milne Circle,1004 Milne Circle,COVINGTON,LA,70435,ST. TAMMANY,Jodie Robinson,jrobinson@milnegh.com,5.0,5,...,NaN,NaN,NaN,"ALEXANDER MILNE DEVELOPMENTAL SERVICES, INC.",NaN,NaN,NaN,NaN,1004 MILNE CIRCLE,70435
2,1008 Milne Circle,1008 Milne Circle,COVINGTON,LA,70435,ST. TAMMANY,Jodie Robinson,jrobinson@milnegh.com,5.0,5,...,NaN,NaN,NaN,"ALEXANDER MILNE DEVELOPMENTAL SERVICES, INC.",NaN,NaN,NaN,NaN,1008 MILNE CIRCLE,70435
3,1012 Milne Circle,1012 Milne Circle,COVINGTON,LA,70435,ST. TAMMANY,Jodie ` Robinson,jrobinson@milnegh.com,4.0,4,...,NaN,NaN,NaN,ALEXANDER MILNE DEVELOPMENTAL SERVICES,NaN,NaN,NaN,NaN,1012 MILNE CIRCLE,70435
4,1016 Milne Circle,1016 Milne Circle,COVINGTON,LA,70435,ST. TAMMANY,Jodie Robinson,jrobinson@milnegh.com,5.0,5,...,NaN,NaN,NaN,Alexander Milne Developmental Services,NaN,NaN,NaN,NaN,1016 MILNE CIRCLE,70435


In [12]:
state_data["NAME_OF_CORPORATION"].value_counts()

NORMAL LIFE OF LAFAYETTE, INC.                        47
DDMS of Louisiana No. 2                               22
EVERGREEN LIFE SERVICES                               20
WESTSIDE HABILITATION CENTER                          19
RES-CARE, INC.                                        16
                                                      ..
STRIVE INCORPORATED                                    1
G.B.COOLEY HOSPITAL SERVICE DISTRICT                   1
Evergreen Presbyterian Ministries                      1
MORNING LIGHT, INC                                     1
VOLUNTEERS OF AMERICA OF GREATER BATON ROUGE, INC.     1
Name: NAME_OF_CORPORATION, Length: 162, dtype: int64

BrightSpring operates under a number of subsidiaries in Louisiana. This checks that we have accounted for all of the different spellings.

In [13]:
state_data.loc[
    lambda x: x["NAME_OF_CORPORATION"].str.contains("NORMAL LIFE") | 
    x["NAME_OF_CORPORATION"].str.contains("RES-", case=False) |
    x["NAME_OF_CORPORATION"].str.contains("COMMUNITY ALTERNATIVES")
]["NAME_OF_CORPORATION"].unique()

array(['NORMAL LIFE OF LOUISIANA', 'Res-Care, Inc.',
       'NORMAL LIFE OF LAFAYETTE, INC', 'NORMAL LIFE OF LOUISIANA, INC.',
       'NORMAL LIFE OF LAFAYETTE, INC.', 'RES-CARE, INC',
       'RES-CARE, INC.', 'NORMAL LIFE OF LAKE CHARLES, INC.',
       'COMMUNITY ALTERNATIVES OF LOUISIANA', 'Res-Care', 'Res-Care, Inc'],
      dtype=object)

In [14]:
brightspring_dbas = [
        "COMMUNITY ALTERNATIVES OF LOUISIANA",
        "NORMAL LIFE OF LAFAYETTE, INC",
        "NORMAL LIFE OF LAFAYETTE, INC.",
        "NORMAL LIFE OF LAKE CHARLES, INC.",
        "NORMAL LIFE OF LOUISIANA",
        "NORMAL LIFE OF LOUISIANA, INC.",
        "Res-Care, Inc.",
        "RES-CARE, INC",
        "RES-CARE, INC.",
        "Res-Care, Inc",
        "Res-Care"
    ]

In [15]:
len(state_data)

579

In [16]:
la_facs = (
    pd
    .read_csv("../../data/qcor/facilities.csv", parse_dates=["termination_date"])
    .loc[lambda x: (x["state"] == "LA") & (x["termination_date"].isnull() | (x["termination_date"] >= "2016-01-01"))]
    .assign(zip_code = lambda df: df["address"].apply(lambda x: x.split()[-1]))
)

In [17]:
la_facs.head()

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,zip_code
2254,NORTH LAKE SERVICES,19G004,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"45439 LIVE OAK DRIVE\nHAMMOND, LA 70401",225 567-7201,1982-09-30,163,No,Non-Profit,"Vol-Merg, Close",2017-01-19,70401
2256,ROSEWOOD HOME,19G006,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"5401 SHED ROAD UNIT 131\nBOSSIER CITY, LA 71111",318 742-6220,1982-08-13,15,No,Non-Profit,NaN,NaT,71111
2257,PINECREST SUPPORTS AND SERVICES CENTER,19G007,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"100 PINECREST DRIVE\nPINEVILLE, LA 71361",318 641-2000,1982-09-23,1222,No,Government,NaN,NaT,71361
2259,ELECTA STREET GROUP HOME,19G009,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"1900 ELECTA STREET\nRAYNE, LA 70578",337 384-6842,1983-01-01,8,No,Non-Profit,NaN,NaT,70578
2260,ROBINSWOOD SCHOOL,19G010,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"4029 AVENUE G\nLAKE CHARLES, LA 70615",337 436-6664,1982-09-21,120,No,For Profit,NaN,NaT,70615


In [18]:
len(la_facs)

573

We merge on the cleaned name and the ZIP Code, which creates a unique match.

In [19]:
merged_data = pd.merge(
    la_facs,
    state_data[["capped_name","zip_code", "NAME_OF_CORPORATION"]],
    how="left",
    left_on=["name","zip_code"],
    right_on=["capped_name","zip_code"]
).drop_duplicates()

In [20]:
len(merged_data)

573

After the merge these 8 homes do not have corporations associated with them. BuzzFeed News did additional research to verify that these homes are not owned by any BrightSpring subsidiary. 

In [21]:
merged_data.loc[
    lambda x: x["NAME_OF_CORPORATION"].isnull()
]

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,zip_code,capped_name,NAME_OF_CORPORATION
89,G B COOLEY CLAY STREET GROUP HOME,19G143,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"200 CLAY STREET\nWEST MONROE, LA 71291",318 325-9704,1984-06-26,8,No,Government,NaN,NaT,71291,NaN,NaN
457,CENTRAL AVENUE GROUP HOME,19G602,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"310 CENTRAL AVENUE\nNEW IBERIA, LA 70560",337 367-1158,2006-09-06,8,No,Non-Profit,NaN,NaT,70560,NaN,NaN
563,JERICHO HOUSE,19G823,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"2005 HUIE-DELLMON AVENUE\nBOYCE, LA 71409",318 445-6443,2021-12-02,8,No,Non-Profit,NaN,NaT,71409,NaN,NaN
564,A B HORN GROUP HOME,19G824,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"1306 MAIN STREET\nBATON ROUGE, LA 70802",225 336-0349,2021-08-25,8,No,Non-Profit,NaN,NaT,70802,NaN,NaN
565,HARMONY TRANSITIONAL SERVICES 3,19G825,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"168-170 WWASHINGTON STREET\nBATON ROUGE, LA 70802",225 256-0086,2021-08-25,15,No,Non-Profit,NaN,NaT,70802,NaN,NaN
567,MAGNOLIA BOLLINGER,19G827,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"749 ST GEORGE AVENUE\nJEFFERSON, LA 70121",504 731-1306,2021-10-13,8,No,Non-Profit,NaN,NaT,70121,NaN,NaN
568,GLENBROOK HOME,19G828,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"5401 SHED ROAD, UNIT 132\nBOSSIER CITY, LA 71111",318 742-6220,2021-10-01,15,No,Non-Profit,NaN,NaT,71111,NaN,NaN
569,WOODBRIAR HOME,19G829,Intermediate Care Facilities for Individuals w...,(VI) Dallas,LA,"5401 SHED ROAD, UNIT 133\nBOSSIER CITY, LA 71111",318 742-6220,2021-11-18,15,No,Non-Profit,NaN,NaT,71111,NaN,NaN


In [22]:
(
    merged_data
    .assign(is_brightspring = lambda df: df["NAME_OF_CORPORATION"].isin(brightspring_dbas))
    .rename(columns={"NAME_OF_CORPORATION": "legal_owner"})[[
        "name", "provider_id", "type", "region", "state", "address",
        "phone", "address", "particip_date", "certified_beds",
        "hospital_based", "ownership_type", "termination_code",
        "termination_date", "legal_owner", "is_brightspring"
    ]]
    .to_csv("../../data/ownership/final/la.csv", index=None)
)

---

---

---